<a href="https://colab.research.google.com/github/joako-m-g/2048IA/blob/main/src/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Set Up

In [2]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from funcs_aux import rolling_sales
from datetime import timedelta


In [6]:
data = pd.read_csv("drive/MyDrive/datos_unidos.csv")

In [4]:
print(data.columns)

Index(['SKU', 'DATE', 'STORE_ID', 'PRICE', 'QUANTITY', 'TOTAL_SALES', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE', 'OPENDATE', 'CLOSEDATE',
       'STORE_SUBGROUP_DATE_ID', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND', 'INITIAL_TICKET_PRICE', 'BASE_PRICE',
       'COSTOS', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE',
       'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'WEEK'],
      dtype='object')


In [5]:
print(data.head())

          SKU        DATE STORE_ID  PRICE  QUANTITY  TOTAL_SALES     REGION  \
0  BEAHASH001  2021-01-01   S00068  35.53        10       355.30       West   
1  BEAHASH001  2021-01-01   S00086  33.52         2        67.04  Southeast   
2  BEAHASH001  2021-01-01   S00124  37.61         2        75.22  Northeast   
3  BEAHASH001  2021-01-01   S00140  34.51         4       138.04  Southeast   
4  BEAHASH001  2021-01-02   S00013  33.77         1        33.77  Southwest   

         CITY STATE STORE_TYPE  ... COSTOS YEAR_OPEN YEAR_CLOSE MONTH_OPEN  \
0    Lakewood    CO    Express  ...  15.07      2018       2025         11   
1     Raleigh    NC    Express  ...  15.07      2011       2025         12   
2        Erie    PA     Outlet  ...  15.07      2018       2025          5   
3  Greenville    SC       Mall  ...  15.07      2011       2025          5   
4     El Paso    TX       Mall  ...  15.07      2019       2025         10   

  MONTH_CLOSE  YEAR MONTH DAY DAY_OF_WEEK  WEEK  
0     

In [18]:
print(len(data))

6825953


## Linear Regression

Enlistamos columnas categoricas y numericas

In [32]:
numericas = [ # No incluimos target (TOTAL_SALES)
    'PRICE', 'COSTOS',
    'INITIAL_TICKET_PRICE', 'BASE_PRICE',
    'YEAR_OPEN', 'MONTH_OPEN',
    'YEAR', 'MONTH', 'DAY', 'WEEK',
    # Las siguientes features se generan dentro del WalkFroward
    #'SKU_mean_7D', 'SKU_std_7D', 'SKU_mean_30D', 'SKU_std_30D', 'SKU_mean_90D', 'SKU_std_90D',
    #'SUBGROUP_mean_7D', 'SUBGROUP_std_7D', 'SUBGROUP_mean_30D', 'SUBGROUP_std_30D', 'SUBGROUP_mean_90D', 'SUBGROUP_std_90D',
    #'STORE_ID_mean_7D', 'STORE_ID_std_7D', 'STORE_ID_mean_30D', 'STORE_ID_std_30D', 'STORE_ID_mean_90D', 'STORE_ID_std_90D'
]
categoricas = [
    'SKU', 'STORE_ID', 'REGION', 'CITY', 'STATE', 'STORE_TYPE', 'OPENDATE', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE', 'PRICE_GROUP_ID', 'BRAND', 'DAY_OF_WEEK'
]

In [33]:
model = Ridge(alpha=1.0)


## WalkForward

In [37]:
def walk_forward_forecast_low_ram(df, model, features, target, train_days=365, step_days=30, forecast_days=7):
    """
    Walk-forward low RAM con rolling interno y manejo seguro de columnas.
    df: DataFrame con columna 'DATE'
    model: modelo sklearn
    features: lista de columnas a usar como features
    target: columna objetivo
    train_days: tamaño inicial del set de entrenamiento
    step_days: incremento de la ventana
    forecast_days: horizonte de predicción
    """
    # --- Asegurar tipo datetime y orden
    df["DATE"] = pd.to_datetime(df["DATE"])
    df = df.sort_values("DATE").reset_index(drop=True)

    results = []
    min_date = df["DATE"].min()
    max_date = df["DATE"].max()
    start_train_end = min_date + timedelta(days=train_days)
    date_values = df["DATE"].values

    # Convertir categóricas a 'category' y codificar
    cat_cols = df.select_dtypes(include='object').columns.tolist()
    for col in cat_cols:
        df[col] = df[col].astype('category').cat.codes

    while start_train_end + timedelta(days=forecast_days) <= max_date:
        train_idx = date_values < start_train_end
        test_idx  = (date_values >= start_train_end) & (date_values < start_train_end + timedelta(days=forecast_days))

        if not test_idx.any() or not train_idx.any():
            start_train_end += timedelta(days=step_days)
            continue

        df_train_window = df.loc[train_idx].copy()
        df_test_window  = df.loc[test_idx].copy()

        # --- Rolling interno solo sobre train
        rolling_sales(df_train_window, group='SKU', windows=[7, 30, 90], std=True)
        rolling_sales(df_train_window, group='SUBGROUP', windows=[7, 30, 90], std=True)
        rolling_sales(df_train_window, group='STORE_ID', windows=[7, 30, 90], std=True)

        # --- Llenar rolling features en test usando últimos valores de train
        rolling_cols = [c for c in df_train_window.columns if 'mean' in c or 'std' in c]
        for col in rolling_cols:
            last_values = df_train_window.groupby('SKU')[col].last()
            df_test_window[col] = df_test_window['SKU'].map(last_values).fillna(0)

        # --- Asegurar que todas las columnas de features existan
        for col in features:
            if col not in df_train_window.columns:
                df_train_window[col] = 0
            if col not in df_test_window.columns:
                df_test_window[col] = 0

        # --- Selección de X e y
        X_train = df_train_window[features]
        y_train = df_train_window[target]
        X_test  = df_test_window[features]
        y_test  = df_test_window[target]

        # --- Reducir cardinalidad de columnas de alta cardinalidad
        cat_high_card = ['SKU', 'STORE_ID']
        min_count_dict = {'SKU': 10, 'STORE_ID': 5}
        X_train, X_test = reduce_cardinality(X_train, X_test, cat_high_card, min_count_dict)

        # --- Entrenamiento y métricas
        model.fit(X_train, y_train)
        r2_train = r2_score(y_train, model.predict(X_train))
        r2_test  = r2_score(y_test, model.predict(X_test))

        results.append({
            "train_end_date": start_train_end,
            "r2_train": r2_train,
            "r2_test": r2_test
        })

        start_train_end += timedelta(days=step_days)

    return pd.DataFrame(results)

Funcion para reduccion de cardinalidad de variables categoricas

In [35]:
# --- Función para reducir cardinalidad
def reduce_cardinality(X_train, X_test, cat_high_card, min_count_dict):
    X_train = X_train.copy()
    X_test = X_test.copy()

    for col in cat_high_card:
        counts = X_train[col].value_counts()
        valid = counts[counts >= min_count_dict[col]].index
        X_train[col] = X_train[col].where(X_train[col].isin(valid), other='Other')
        X_test[col] = X_test[col].where(X_test[col].isin(valid), other='Other')
    return X_train, X_test



Hacemos una walkforward para evaluar estabilidad del modelo y evitar sobreajuste.

In [39]:
results_walk = walk_forward_forecast_low_ram(
    data,
    model,
    features = categoricas + numericas,
    target='QUANTITY'
)


In [40]:
print(results_walk)

   train_end_date  r2_train   r2_test
0      2022-01-01  0.052431 -0.021940
1      2022-01-31  0.053059 -0.006463
2      2022-03-02  0.051717 -0.139007
3      2022-04-01  0.057660 -0.139382
4      2022-05-01  0.062536 -0.041970
5      2022-05-31  0.065102 -0.343101
6      2022-06-30  0.065497 -0.022594
7      2022-07-30  0.063268 -0.013935
8      2022-08-29  0.060438 -0.012503
9      2022-09-28  0.058493  0.010222
10     2022-10-28  0.057980  0.029997
11     2022-11-27  0.058603 -0.100588
12     2022-12-27  0.057849 -0.154394
13     2023-01-26  0.057233 -0.004745
14     2023-02-25  0.062742 -0.018949
15     2023-03-27  0.070054 -0.070399
16     2023-04-26  0.076861 -0.028124
17     2023-05-26  0.081357 -0.195130
18     2023-06-25  0.084419 -0.008845
19     2023-07-25  0.083849 -0.025302
20     2023-08-24  0.080560  0.001188
21     2023-09-23  0.078929  0.007238
22     2023-10-23  0.077313  0.024401
23     2023-11-22  0.076144 -0.126470
24     2023-12-22  0.075040 -0.101300


In [44]:
# Supongamos que results_walk es el DataFrame con tus resultados
results_walk.to_csv("linear_model_walk_results.csv", index=False)
